*This notebook requires Python 3.6*

# Comparing return level wind speeds for different GPD fitting methods

To make full use of the scenarios generated in TCRM, it is preferred to use a Generalised Pareto Distribution (GPD) to fit the extremes of the distribution of simulated wind speeds. This utilises the peaks-over-threshold (POT) approach, rather than the block maxima (BM) approach used for the Generalised Extreme Value (GEV) distribution.

The Generalised Pareto Distribution (GPD) is defined as:

$H(y) = 1 - (1 + \xi y / \check{s}) ^{-1/\xi}$

where $\check{s} = \sigma + \xi(u - \mu)$ and $u = $ threshold value. $\sigma$ and $\mu$ are the scale and location parameters of a corresponding GEV distribution. If the data can be fitted to a GEV distribution, then values above the threshold can be fitted with a GPD.

The fit of the GPD is highly sensitive to the selected threshold in POT methods. Too low a threshold and the fit is biased towards the bulk distribution at lower values. Too high a threshold and there are too few data points to provide a stable fit. Further to this, we want to select a threshold that results in a GPD with a negative shape parameter so that the resulting return levels are bounded. 

Here, we have precalculated the parameters for a GPD from the simulated TC wind speeds at a large number of locations across Australia. We used the iterative threshold selection method described by Sanabria and Cechet (2007), and a maximum likelihood estimation method, implemented in the `scipy.stats` package, using an arbitrary threshold (in our case, the 99.5th percentile of the simulated wind speeds at the site). We compare the return levels using these two methods, with a view to implementing one or the other in the TCRM code.

In [ ]:
%matplotlib notebook

import os
import sys
from os.path import join as pjoin

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

sns.set_context('notebook')
sns.set_style('darkgrid')

We start with loading the data. This includes the return levels calculated using the iterative threshold selection and the percentile threshold selection, and the distribution parameters for each method. You can use the [tcrmextremes.py](https://github.com/wcarthur/extremes/blob/python/tcrmextremes.py) script to build these data files. This relies on a completed simulation of TCRM for the region you are investigating.

In [ ]:
inputPath = "C:/WorkSpace/data/derived/tc/tcha/"
iterative_rl_file = pjoin(inputPath, "iterative_rl.csv")
fitted_rl_file = pjoin(inputPath, "fitted_rl.csv")
parameter_file = pjoin(inputPath, "parameters.csv")
names = ['locId', 'locName', '1', '2', '5', '10', '20', '50',
         '100', '200', '500', '1000', '2000', '5000', '10000']
usecols = ['locId', '1', '2', '5', '10', '20', '50', '100',
           '200', '500', '1000', '2000', '5000', '10000']

it = pd.read_csv(iterative_rl_file, names=names, index_col='locId',
                 usecols=usecols, header=0)
ft = pd.read_csv(fitted_rl_file, names=names, index_col='locId',
                 usecols=usecols, header=0)

paramnames = ['locId', 'locName', 'it_scale', 'it_shape', 'it_thresh', 'it_rate',
              'gpd_rate', 'gpd_shape', 'gpd_thresh', 'gpd_scale']
usecols = ['locId', 'locName', 'it_scale', 'it_shape', 'it_thresh', 'it_rate',
           'gpd_rate', 'gpd_shape', 'gpd_thresh', 'gpd_scale']
params = pd.read_csv(parameter_file, names=paramnames, index_col='locId', usecols=usecols, header=0)

In [ ]:
ft.head(20)

In [ ]:
it.head(20)

In [ ]:
params.head(20)

In [ ]:
params = params[~params.duplicated()]
it = it[~it.index.duplicated()]
ft = ft[~ft.index.duplicated()]

We need to filter the data, to remove those records with an undefined shape parameter, a threshold of zero (which indicates a location with very few data points), and positive return level values for all return periods. This ensures the statistics we inspect are not influenced by questionable data. The replacement of invalid values is performed in-place.

In [ ]:
idx = params.where((params['it_shape'] !=0.0 ) & (params['gpd_shape'] != 0.0) &\
                   (params['gpd_thresh'] != 0.0) & (ft['1'] > 0.0) & (it['1'] > 0.0))
flag = idx['it_shape'].isnull()
it.loc[flag] = np.nan
ft.loc[flag] = np.nan

Take the difference between the two data frames. The resulting values are positive where the iterative selection method gives higher return levels, and negative where the percentile selection method gives higher return values.

In [ ]:
diff = it.rsub(ft, axis=0)
diff

In [ ]:
diff.mean()

In [ ]:
diff.std()

Plot these data in a simple box plot to view the results.

In [ ]:
diff.plot.box(title="Difference in return level wind speeds (n={0})".format(diff["1"].notnull().sum()))

So, with 319 valid sites, the mean difference between the two methods is at most 2.8 m/s. For the return periods that we are interested in for extreme events (> 50 years), the mean difference between the two methods is no more than 1 m/s. 

In terms of computing return levels quickly, the percentile threshold selection method is far superior. Given the computational efficiency, and the minimal differences in return levels (especially at long return periods), the percentile threshold selection method is being implemented into TCRM.

In [ ]:
rls = list(diff.columns.values)

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as feature
import cartopy.io
import geopandas as gpd

In [ ]:
locationFilePath = "C:/WorkSpace/data/derived/tcobs/merged.shp"
locdf = gpd.read_file(locationFilePath)
locdf = locdf.set_index(["locId"])
locdf.head(20)

In [ ]:
joindf = pd.merge(locdf, diff, left_index=True, right_index=True )

In [ ]:
finaldf = joindf[joindf['1'].notnull()]

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.add_feature(feature.BORDERS)
cs = ax.scatter(x=finaldf['Longitude'], y=finaldf['Latitude'], c=finaldf['100'], s=50,
           cmap=sns.diverging_palette(240, 10, n=9, center='light', as_cmap=True))
gl = ax.gridlines(linestyle=":",color='k', draw_labels=True)
plt.colorbar(cs, orientation='horizontal')
plt.show()

In [ ]:
def getReturnLevelDiff(rl, ):
    df = diff.dropna([rl])
    for locId in diff.index.values:
        if diff.loc[locId][rl]:
            output.append()
        print(diff.loc[locId][rl])

In [ ]:
df = diff[diff["1"].notnull()]
df.head(10)